In [24]:
import re
import math
from functools import cached_property

In [32]:
class Maze(object):
    DVEC = {
        0: (0,1),
        1: (1,0),
        2: (0,-1),
        3: (-1,0),
    }

    def __init__(self,file,start_direction = 0,geometry='plane') -> None:
        self._file = file
        self._start_direction = start_direction
        self._maze, self._size, self._moves, self._start = self._load_input()
        self._steps = [self._start]
        self._geometry = geometry
        self.trace()

    @cached_property
    def qs(self):
        return int(math.sqrt(len(self._maze)/6))

    @cached_property
    def edges(self):
        edges = {}

        edges.update({((0,x),3):((y,0),0) for x,y in zip(range(self.qs*1,self.qs*2),range(self.qs*3,self.qs*4))})
        edges.update({((0,x),3):((self.qs*4-1,y),3) for x,y in zip(range(self.qs*2,self.qs*3),range(self.qs*0,self.qs*1))})
        edges.update({((x,self.qs*3-1),0):((y,self.qs*2-1),2) for x,y in zip(range(self.qs*0,self.qs*1),reversed(range(self.qs*2,self.qs*3)))})
        edges.update({((self.qs*1-1,x),1):((y,self.qs*2-1),2) for x,y in zip(range(self.qs*2,self.qs*3),range(self.qs*1,self.qs*2))})
        edges.update({((self.qs*3-1,x),1):((y,self.qs*1-1),2) for x,y in zip(range(self.qs*1,self.qs*2),range(self.qs*3,self.qs*4))})
        edges.update({((x,self.qs*1),2):((self.qs*2,y),1) for x,y in zip(range(self.qs*1,self.qs*2),range(self.qs*0,self.qs*1))})
        edges.update({((x,self.qs*1),2):((y,0),0) for x,y in zip(range(self.qs*0,self.qs*1),reversed(range(self.qs*2,self.qs*3)))})

        # edges.update({((0,x),3):((4,y),1) for x,y in zip(range(8,12),reversed(range(0,4)))})
        # edges.update({((x,8),2):((4,y),1) for x,y in zip(range(0,4),range(4,8))})
        # edges.update({((x,11),0):((y,15),2) for x,y in zip(range(0,4),reversed(range(8,12)))})
        # edges.update({((x,11),0):((8,y),1) for x,y in zip(range(4,8),reversed(range(12,16)))})
        # edges.update({((7,x),1):((y,8),0) for x,y in zip(range(4,8),reversed(range(8,12)))})   
        # edges.update({((7,x),1):((11,y),3) for x,y in zip(range(0,4),reversed(range(8,12)))})
        # edges.update({((x,0),2):((11,y),3) for x,y in zip(range(4,8),reversed(range(12,16)))})

        edges.update({(v[0],(v[1]-2) % 4):(k[0],(k[1]-2) % 4 ) for k,v in edges.items()})

        return edges


    def _load_input(self):
        with open(self._file, 'r') as file:
            input = [line for line in file.read().splitlines() if line != '']
        
        
        movements = input.pop(-1)
        size = (len(input), max([len(x) for x in input]))

        board = {}

        for r,line in enumerate(input):
            for c in range(size[1]):
                if (c < len(line)) and (v := line[c]) != ' ':
                    board[(r,c)] = v
        

        steps = list(map(int,re.split(r"[RL]",movements)))
        rotations = [x for x in re.split(r"\d",movements) if x != '']
        moves = [x for y  in zip(steps,rotations) for x in y]
        if len(steps) + len(rotations) > len(moves):
            moves.append(steps[-1])

        start = [((0,c),self._start_direction) for c in range(size[1]) if board.get((0,c))][0]

        return board, size, moves, start

    
    def rotate(self,position,direction):
        # > = 0
        # v = 1
        # < = 2
        # ^ = 3

        match direction:
            case "L":
                return (position[0],(position[1] - 1) % 4)
            case "R":
                return (position[0],(position[1] + 1) % 4)
    
    def move(self,position,steps):


        direction = position[1]
        temp_pos = position[0]
        pos = temp_pos
        i = 0
        
        while i < steps:
            if self._geometry == 'cube' and ((temp_pos,direction) in self.edges):
                next_pos, next_direction = self.edges[(temp_pos,direction)]
            else:
                next_pos = tuple([x+y for x,y in zip(temp_pos,self.DVEC[direction])])
                next_pos = (next_pos[0] % self._size[0], next_pos[1] % self._size[1])
                next_direction = direction

            if v := self._maze.get(next_pos):
                if v == '.':
                    i+=1
                    pos = next_pos
                elif v == '#':
                    break
                else:
                    continue
            
            temp_pos = next_pos
            direction = next_direction
        return (pos, direction)

    def trace(self):
        for move in self._moves:
            if isinstance(move,int):
                step = self.move(self._steps[-1],move)
            elif move in ['L','R']:
                step = self.rotate(self._steps[-1],move)
            else:
                raise ValueError(f"unrecognized move: {move}")

            self._steps.append(step)


    @property
    def score(self):
        return ((self._steps[-1][0][0] + 1) * 1000 + ((self._steps[-1][0][1] + 1) * 4 + self._steps[-1][1]))


In [33]:
pt1 = Maze('./assets/input_day_22.txt',geometry='plane')
pt1.score

50412

In [34]:
pt2 = Maze('./assets/input_day_22.txt',geometry='cube')
pt2.score

130068